In [1]:
!pip install transformers
!pip install numpy==1.26.0
!pip install tensorflow[and-cuda]
!pip install torch torchvision torchaudio
!pip install --upgrade ipywidgets
!pip install tf-keras
!pip install pandas
!pip install scikit-learn
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 1.26.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:0

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Data Preprocessing

In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("data/train_data.csv")
test_data = pd.read_csv("data/test_data.csv")

In [3]:
POSITIVE_WORDS = ["positive", "great", "good", "happy", "amazing", "fantastic", "yes"]
NEGATIVE_WORDS = ["negative", "bad", "sad", "terrible", "horrible", "no", "critical"]

In [4]:
def convert_lbl_to_int(label):
    if label.lower() in POSITIVE_WORDS:
        return 1
    if label.lower() in NEGATIVE_WORDS:
        return 0
    return -1

## Baseline Model

In [5]:
import numpy as np
print(np.__version__)
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from sklearn.metrics import f1_score
from torchinfo import summary
import random

1.26.0


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-02-25 07:07:08.711159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740467228.733313  142683 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740467228.740137  142683 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 07:07:08.764117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable

In [6]:
def get_prompt(review):
  with open("baseline_prompt.txt", "r") as file:
    prompt = file.read()
  prompt = prompt.replace("<REVIEW>", review)
  return prompt

print(get_prompt("Test Prompt"))

Given a book review, classify it as expressing a positive or negative sentiment.
Review: Test Prompt
This review is either positive or negative sentiment. If one had to chosen, the sentiment in the review is [MASK].



In [7]:
# Load the BERT model and tokenizer
baseline_model = BertModel.from_pretrained("bert-base-uncased")
baseline_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
baseline_unmasker = pipeline('fill-mask', model='bert-base-uncased')

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [8]:
def predict(review, unmasker):
  prompt = get_prompt(review)
  predictions = unmasker(prompt)
  valid_predictions = POSITIVE_WORDS + NEGATIVE_WORDS
    
  for prediction in predictions:
    if(prediction['token_str'] in valid_predictions):
      return prediction['token_str']

  for prediction in predictions:
    print(prediction['token_str'])
        
  sentiment = ["positive", "negative"]
  #If not found, lets predict random
  return "NOT FOUND" #random.choice(sentiment)

In [9]:
def predict_baseline(df, model, tokenizer, unmasker):
    predictions = []
    labels = []
    for idx, row in df.iterrows():
        input_text = row['Review']
        prediction = predict(input_text, unmasker)
        predictions.append(convert_lbl_to_int(prediction))
        labels.append(convert_lbl_to_int(row['Sentiment']))
    return predictions, labels



In [10]:
def score_baseline(baseline_model, df, baseline_tokenizer, baseline_unmasker):
    predictions, labels = predict_baseline(df, baseline_model, baseline_tokenizer, baseline_unmasker)
    values, counts = np.unique(np.array(predictions), return_counts=True)
    
    for v, c in zip(values, counts):
        print(f"Value: {v}, Count: {c}")

    return f1_score(labels, predictions, average='micro')    
    

In [11]:
train_baseline_f1 = score_baseline(baseline_model, train_data, baseline_tokenizer, baseline_unmasker)
print(train_baseline_f1)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ridiculous
wrong
silly
right
absurd
Value: -1, Count: 1
Value: 0, Count: 4542
Value: 1, Count: 3457
0.67075


In [12]:
test_baseline_f1 = score_baseline(baseline_model, test_data, baseline_tokenizer, baseline_unmasker)
print(test_baseline_f1)

Value: 0, Count: 1143
Value: 1, Count: 857
0.6425


## Probe Models

In [13]:
import torch.nn as nn
import torch
from transformers import AutoTokenizer, BertModel
import pandas as pd

class CustomBERTModel(nn.Module):
    def __init__(self, bert_embedding_layer, bert_encoder_layers):
        super(CustomBERTModel, self).__init__()
        self.bert_embedding_layer = bert_embedding_layer
        self.bert_encoder_layers = bert_encoder_layers
        self.dropout = nn.Dropout(0.5)
        
        # Freeze the embedding layer
        for param in self.bert_embedding_layer.parameters():
            param.requires_grad = False

        for param in self.bert_encoder_layers.parameters():
            param.requires_grad = False
        
        ### New layers:
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(768, 256)
        self.linear2 = nn.Linear(256, 1)
        self.linear3 = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        embedding_output = self.bert_embedding_layer(ids)
        encoder_outputs = embedding_output
        for i, layer_module in enumerate(self.bert_encoder_layers):
          encoder_outputs = layer_module(encoder_outputs)[0]

        linear1_output = self.dropout(self.linear1(encoder_outputs))
        linear2_output = self.dropout(self.linear2(linear1_output))
        flattened_output = self.flatten(linear2_output)
        linear3_output = self.dropout(self.linear3(flattened_output))
        sigmoid_output = self.sigmoid(linear3_output)
        return sigmoid_output


In [14]:
def get_custom_bert_model(num_bert_layers):
    return CustomBERTModel(baseline_model.embeddings, baseline_model.encoder.layer[:num_bert_layers]).to(torch.device("cuda"))

In [15]:
summary(get_custom_bert_model(4))

Layer (type:depth-idx)                             Param #
CustomBERTModel                                    --
├─BertEmbeddings: 1-1                              --
│    └─Embedding: 2-1                              (23,440,896)
│    └─Embedding: 2-2                              (393,216)
│    └─Embedding: 2-3                              (1,536)
│    └─LayerNorm: 2-4                              (1,536)
│    └─Dropout: 2-5                                --
├─ModuleList: 1-2                                  --
│    └─BertLayer: 2-6                              --
│    │    └─BertAttention: 3-1                     (2,363,904)
│    │    └─BertIntermediate: 3-2                  (2,362,368)
│    │    └─BertOutput: 3-3                        (2,361,600)
│    └─BertLayer: 2-7                              --
│    │    └─BertAttention: 3-4                     (2,363,904)
│    │    └─BertIntermediate: 3-5                  (2,362,368)
│    │    └─BertOutput: 3-6                        (2,361,6

In [16]:
# bert_seq_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# print(inputs)
# with torch.no_grad():
#     logits = bert_seq_model(**inputs).logits
#     print(logits.shape)

# predicted_class_id = logits.argmax().item()
# print(bert_seq_model.config.id2label[predicted_class_id])

# summary(bert_seq_model)

In [17]:
def get_inputs_labels(df):
    inputs = []
    labels = []
    for idx, row in train_data.iterrows():
        input_text = row['Review']
        inputs.append(input_text)
        labels.append(1 if row['Sentiment'] == 'positive' else 0)
    return inputs, labels


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

In [19]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
inputs, labels = get_inputs_labels(train_data)
dataset = TextDataset(inputs, labels, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [20]:
def predict_custom(model, df):
    model.eval()
    predictions = []
    labels = []
    with torch.no_grad():
        for idx, row in df.iterrows():
            input_text = row['Review']
            encoding = tokenizer(input_text, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True, return_tensors='pt')
            input_ids = encoding['input_ids'].cuda()
            attention_mask = encoding['attention_mask'].cuda()
            prediction = model(encoding['input_ids'].cuda(), encoding['attention_mask'].cuda())
            predictions.append(1 if prediction > 0.5 else 0)
            labels.append(convert_lbl_to_int(row['Sentiment']))

    return predictions, labels

In [21]:
def score_custom_model(model, train_df, test_df):
    train_f1 = None
    test_f1 = None
    
    if train_df is not None:
        train_predictions, train_labels = predict_custom(model, train_df)
        train_f1 = f1_score(train_labels, train_predictions, average='micro')

    if test_df is not None:
        test_predictions, test_labels = predict_custom(model, test_df)
        test_f1 = f1_score(test_labels, test_predictions, average='micro')
        
    return train_f1, test_f1 
    
# print(score_custom_model(model, train_data, test_data))

In [22]:
def get_loss(model, df):
    inputs, labels = get_inputs_labels(df)
    

In [23]:
def train_custom_model(model, epochs, dataloader, train_df):
    criterion = nn.BCELoss() ## If required define your own criterion
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    for epoch in range(epochs):
        for batch in dataloader: 
            targets = np.array(batch['label'])
            targets = torch.tensor(np.expand_dims(targets,axis=1)).float().cuda()
            optimizer.zero_grad()
            input_ids = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            outputs = model(input_ids.cuda(), attention_mask.cuda())
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        train_f1, _ = score_custom_model(model, train_df, None)
        print("Epoch: " + str(epoch) + " F1: " + str(train_f1))

In [24]:
def score_all_probe_models(dataloader, train_df, test_df, epochs, max_n):
    model_scores = []
    for n in range(1,max_n + 1):
        print("N: " + str(n))
        model = get_custom_bert_model(n)
        train_custom_model(model, epochs, dataloader, train_df)
        _, test_f1 = score_custom_model(model, None, test_df)
        print("TEST F1: " + str(test_f1))
        model_scores.append([n, test_f1])

    return model_scores        

In [25]:
model_scores = score_all_probe_models(dataloader, train_data, test_data, 10, 10)

N: 1
Epoch: 0 F1: 0.5945
Epoch: 1 F1: 0.67325
Epoch: 2 F1: 0.79275
Epoch: 3 F1: 0.82725
Epoch: 4 F1: 0.82725
Epoch: 5 F1: 0.832
Epoch: 6 F1: 0.855
Epoch: 7 F1: 0.8625
Epoch: 8 F1: 0.85375
Epoch: 9 F1: 0.860625
TEST F1: 0.8035
N: 2
Epoch: 0 F1: 0.566125
Epoch: 1 F1: 0.64375
Epoch: 2 F1: 0.70975
Epoch: 3 F1: 0.786625
Epoch: 4 F1: 0.794625
Epoch: 5 F1: 0.816625
Epoch: 6 F1: 0.82925
Epoch: 7 F1: 0.782375
Epoch: 8 F1: 0.843
Epoch: 9 F1: 0.828
TEST F1: 0.7795
N: 3
Epoch: 0 F1: 0.5995
Epoch: 1 F1: 0.673125
Epoch: 2 F1: 0.744875
Epoch: 3 F1: 0.80225
Epoch: 4 F1: 0.8225
Epoch: 5 F1: 0.8115
Epoch: 6 F1: 0.825125
Epoch: 7 F1: 0.8435
Epoch: 8 F1: 0.830125
Epoch: 9 F1: 0.854
TEST F1: 0.8205
N: 4
Epoch: 0 F1: 0.50025
Epoch: 1 F1: 0.604875
Epoch: 2 F1: 0.670375
Epoch: 3 F1: 0.749
Epoch: 4 F1: 0.79825
Epoch: 5 F1: 0.801375
Epoch: 6 F1: 0.8005
Epoch: 7 F1: 0.7945
Epoch: 8 F1: 0.8415
Epoch: 9 F1: 0.847
TEST F1: 0.805
N: 5
Epoch: 0 F1: 0.62425
Epoch: 1 F1: 0.71025
Epoch: 2 F1: 0.767375
Epoch: 3 F1: 0.841

## Analysis

In [26]:
summary(baseline_model)

Layer (type:depth-idx)                                  Param #
BertModel                                               --
├─BertEmbeddings: 1-1                                   --
│    └─Embedding: 2-1                                   (23,440,896)
│    └─Embedding: 2-2                                   (393,216)
│    └─Embedding: 2-3                                   (1,536)
│    └─LayerNorm: 2-4                                   (1,536)
│    └─Dropout: 2-5                                     --
├─BertEncoder: 1-2                                      --
│    └─ModuleList: 2-6                                  --
│    │    └─BertLayer: 3-1                              (7,087,872)
│    │    └─BertLayer: 3-2                              (7,087,872)
│    │    └─BertLayer: 3-3                              (7,087,872)
│    │    └─BertLayer: 3-4                              (7,087,872)
│    │    └─BertLayer: 3-5                              (7,087,872)
│    │    └─BertLayer: 3-6            

In [36]:
summary( get_custom_bert_model(10))

Layer (type:depth-idx)                             Param #
CustomBERTModel                                    --
├─BertEmbeddings: 1-1                              --
│    └─Embedding: 2-1                              (23,440,896)
│    └─Embedding: 2-2                              (393,216)
│    └─Embedding: 2-3                              (1,536)
│    └─LayerNorm: 2-4                              (1,536)
│    └─Dropout: 2-5                                --
├─ModuleList: 1-2                                  --
│    └─BertLayer: 2-6                              --
│    │    └─BertAttention: 3-1                     (2,363,904)
│    │    └─BertIntermediate: 3-2                  (2,362,368)
│    │    └─BertOutput: 3-3                        (2,361,600)
│    └─BertLayer: 2-7                              --
│    │    └─BertAttention: 3-4                     (2,363,904)
│    │    └─BertIntermediate: 3-5                  (2,362,368)
│    │    └─BertOutput: 3-6                        (2,361,6